# Bidirectional Encoder Representations from Transformers (BERT)


---

## 目的
Bidirectional Encoder Representations from Transformers (BERT) を理解する．



## モジュールのインポート・データのダウンロード
演習に使用するモジュールとデータをダウンロードします．

In [ ]:
!pip install transformers fugashi ipadic

In [38]:
import os
import urllib.request
import numpy as np
import pandas as pd
import torch
import re
import sys
import csv
import tarfile

# GPUが使えれば利用する設定
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## データセットの作成

ダウンロードしたデータを読み込んで，学習データの作成を行います．

今回使用するデータは，[livedoorニュースコーパスデータセット](https://www.rondhuit.com/download.html#ldcc)です．
これは，Web上のweb記事のデータセットとなっており，複数のカテゴリのニュース記事から構成されています．
今回はこのうち，「ITLife Hack（IT関連記事）」と「Sports Watch（スポーツ関連記事）」を分類するタスクおよびデータセットを扱います．

まずデータをダウンロードし，実験に使用するデータ飲みを抽出し，テキストファイルに一度書き出します．

In [4]:
tgz_fname = "ldcc-20140209.tar.gz"              # ダウンロードした圧縮ファイルのパスを設定
target_genre = ["it-life-hack", "sports-watch"] # 2つをニュースメディアのジャンルを選定
tsv_fname = "all_text.tsv"                      # 処理をした結果を保存するファイル名 

# データのダウンロード（カレントディレクトリに圧縮ファイルがダウンロードされる）
urllib.request.urlretrieve("https://www.rondhuit.com/download/ldcc-20140209.tar.gz", "ldcc-20140209.tar.gz")

# 処理部分 -------
brackets_tail = re.compile('【[^】]*】$')
brackets_head = re.compile('^【[^】]*】')

def remove_brackets(inp):
    output = re.sub(brackets_head, '', re.sub(brackets_tail, '', inp))
    return output

def read_title(f):
    # 2行スキップ
    next(f)
    next(f)
    title = next(f) # 3行目を返す
    title = remove_brackets(title.decode('utf-8'))
    return title[:-1]

zero_fnames = []
one_fnames = []

with tarfile.open(tgz_fname) as tf:
    # 対象ファイルの選定
    for ti in tf:
        # ライセンスファイルはスキップ
        if "LICENSE.txt" in ti.name:
            continue
        if target_genre[0] in ti.name and ti.name.endswith(".txt"):
            zero_fnames.append(ti.name)
            continue
        if target_genre[1] in ti.name and ti.name.endswith(".txt"):
            one_fnames.append(ti.name)
    with open(tsv_fname, "w") as wf:
        writer = csv.writer(wf, delimiter='\t')
        # ラベル 0
        for name in zero_fnames:
            f = tf.extractfile(name)
            title = read_title(f)
            row = [target_genre[0], 0, '', title]
            writer.writerow(row)
        # ラベル 1
        for name in one_fnames:
            f = tf.extractfile(name)
            title = read_title(f)
            row = [target_genre[1], 1, '', title]
            writer.writerow(row)

### データの表示と確認

抽出したデータを読み込み，一部を表示して内容を確認します．

表示すると，文章と記事のクラスなどがまとめられていることが確認できます．

In [5]:
# データの読み込み
df = pd.read_csv("all_text.tsv", 
                 delimiter='\t', header=None, names=['media_name', 'label', 'NaN', 'sentence'])

# データの確認
print(f'データサイズ： {df.shape}')
df.sample(10)

データサイズ： (1770, 4)


,media_name,label,NaN,sentence
1536,sports-watch,1,NaN,山崎明かす、予告ホームランは「5回くらいやってます」
547,it-life-hack,0,NaN,いつでもどこでも自分専用環境！Windowsトラブル時にも使えるUbuntu
1147,sports-watch,1,NaN,浅尾美和、新パートナーとはやくもちぐはぐ!?
1647,sports-watch,1,NaN,巨人の拙劣な采配に批判殺到、ノムさんも「根拠がサッパリわからない」
200,it-life-hack,0,NaN,アイコンを変更してフォルダーを目立たせる
1488,sports-watch,1,NaN,オリックス公式アカウント「ヘディング脳」発言が波紋
1542,sports-watch,1,NaN,ザック・ジャパンの″秘密兵器″報道に賛否
1649,sports-watch,1,NaN,原監督の采配皮肉る落合氏にファンも同調
846,it-life-hack,0,NaN,高ワットを安定供給できる電源！ ZALMANより1250ワット電源発売開始
1334,sports-watch,1,NaN,澤、今後を語る「指導者は絶対ないですね」


### データの抽出

上記では，pandasのDataFrameでデータを読み込んでいるため，
必要なデータのみをDataFrameから抽出します．

In [30]:
# データの抽出
sentences = df.sentence.values
labels = df.label.values

### 日本語文章の分解とIDへの変換

次に，データセットの日本語文章を単語に分解し，各単語に対応するIDへ変換を行います

この処理には，`BertJapaneseTokenizer`を用いて変換します．

まず，`BertJapaneseTokenizer`のインスタンスを作成します．

In [16]:
from transformers import BertJapaneseTokenizer
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

次に作成したTokenizerを使用して，日本語文章を単語およびIDへの変換結果を確認してみます．

単語に分解されていることが確認できます．

In [31]:
# 元文章
print('Original: ', sentences[0])
# Tokenizer
print('Tokenized: ', tokenizer.tokenize(sentences[0]))
# Token-id
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

Original:  旧式Macで禁断のパワーアップ！最新PCやソフトを一挙にチェック
Tokenized:  ['旧式', 'Mac', 'で', '禁', '##断', 'の', 'パワーアップ', '!', '最新', 'PC', 'や', 'ソフト', 'を', '一挙', 'に', 'チェック']
Token IDs:  [18718, 8653, 12, 1763, 29135, 5, 20734, 679, 6215, 3794, 49, 1604, 11, 24598, 7, 9398]


次にデータセット全ての文章を単語へ分解しID変換したデータを作成します．

In [35]:
# 最大単語数の確認
max_len = []

# 1文づつ処理
for sent in sentences:
    # Tokenizeで分割
    token_words = tokenizer.tokenize(sent)
    # 文章数を取得してリストへ格納
    max_len.append(len(token_words))

# 最大の値を確認
print('最大単語数:', max(max_len))
print('注意：上記の最大単語数にSpecial token（[CLS], [SEP]）の+2をした値が最大単語数')

input_ids = []
attention_masks = []

# 1文づつ処理
for sent in sentences:
    encoded_dict = tokenizer.encode_plus(
                        sent,                      
                        add_special_tokens = True,      # Special Tokenの追加
                        max_length = 37,                # 文章の長さを固定（Padding/Trancatinating）
                        pad_to_max_length = True,       # PADDINGで埋める
                        return_attention_mask = True,   # Attention maksの作成
                        return_tensors = 'pt',          #  Pytorch tensorsで返す
                   )

    # 単語IDを取得    
    input_ids.append(encoded_dict['input_ids'])

    # Attention maskの取得
    attention_masks.append(encoded_dict['attention_mask'])

# リストに入ったtensorを縦方向（dim=0）へ結合
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

# tenosor型に変換
labels = torch.tensor(labels)

最大単語数:  35
注意：上記の最大単語数にSpecial token（[CLS], [SEP]）の+2をした値が最大単語数


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:40: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


### データセットクラスの作成

読み込み・整理をしたデータを用いて，データセットクラスを作成します．

In [36]:
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# データセットクラスの作成
dataset = TensorDataset(input_ids, attention_masks, labels)

# 90%地点のIDを取得
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# データセットを分割
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('訓練データ数: {}'.format(train_size))
print('検証データ数: {}'.format(val_size))

# データローダーの作成
batch_size = 32

# 訓練データローダー
train_dataloader = DataLoader(
            train_dataset,  
            sampler = RandomSampler(train_dataset),   # ランダムにデータを取得してバッチ化
            batch_size = batch_size
        )

# 検証データローダー
validation_dataloader = DataLoader(
            val_dataset, 
            sampler = SequentialSampler(val_dataset), # 順番にデータを取得してバッチ化
            batch_size = batch_size
        )

訓練データ数: 1593
検証データ数: 177


## ネットワークモデルの作成

ネットワークモデルを作成します．

今回は，BERTの日本語用のPre-trainedモデルを活用し，Fine-Tuningを行うことで，記事の分類を行います．

Pre-trainedモデルの読み込みには`BertForSequenceClassification`を活用します．

In [37]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# BertForSequenceClassification 学習済みモデルのロード
model = BertForSequenceClassification.from_pretrained(
    "cl-tohoku/bert-base-japanese-whole-word-masking", # 日本語Pretrainedモデルの指定
    num_labels = 2,                                    # ラベル数（今回はBinayなので2、数値を増やせばマルチラベルも対応可）
    output_attentions = False,                         # アテンションベクトルを出力するか
    output_hidden_states = False,                      # 隠れ層を出力するか
)

# モデルをGPUへ転送
model.cuda()

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

## 学習
ネットワークを学習（Fine-Tuning）します．

最適化手法には，Adamに用いるweight decayを改良した`AdamW`を使用します．

In [22]:
# 最適化手法の設定
optimizer = AdamW(model.parameters(), lr=2e-5)

# 訓練パートの定義
def train(model):
    model.train() # 訓練モードで実行
    train_loss = 0
    for batch in train_dataloader:# train_dataloaderはword_id, mask, labelを出力する点に注意
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        optimizer.zero_grad()
        
        loss = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels).loss
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        train_loss += loss.item()
    return train_loss

# テストパートの定義
def validation(model):
    model.eval()# 訓練モードをオフ
    val_loss = 0
    with torch.no_grad(): # 勾配を計算しない
        for batch in validation_dataloader:
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            with torch.no_grad():
                loss = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask,labels=b_labels).loss
            val_loss += loss.item()
    return val_loss

# 学習の実行
max_epoch = 4
train_loss_ = []
test_loss_ = []

for epoch in range(max_epoch):
    train_ = train(model)
    test_ = train(model)
    train_loss_.append(train_)
    test_loss_.append(test_)

## 検証

学習したモデルを検証データで評価します．

In [45]:
# 検証方法の確認（1バッチ分で計算ロジックに確認）
count = 0
model.eval()# 訓練モードをオフ
for batch in validation_dataloader:
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    with torch.no_grad():   
        # 学習済みモデルによる予測結果をpredsで取得     
        preds = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        if count == 0:
            PRED = preds[0]
            LABEL = b_labels
        else:
            PRED = torch.cat((PRED, preds[0]), dim=0)
            LABEL = torch.cat((LABEL, b_labels), dim=0)
        count += 1
        

# 比較しやすい様にpd.dataframeへ整形
# pd.dataframeへ変換（GPUに乗っているTensorはgpu->cpu->numpy->dataframeと変換）
logits_df = pd.DataFrame(PRED.cpu().numpy(), columns=['logit_0', 'logit_1'])

# np.argmaxで大き方の値を取得
pred_df = pd.DataFrame(np.argmax(PRED.cpu().numpy(), axis=1), columns=['pred_label'])
label_df = pd.DataFrame(LABEL.cpu().numpy(), columns=['true_label'])

print("accuracy: ", np.sum(np.argmax(PRED.cpu().numpy(), axis=1) == LABEL.cpu().numpy()) / len(PRED))

accuracy:  0.4745762711864407


最後に結果をDataFrameで一覧表示し，確認します．

In [44]:
accuracy_df = pd.concat([logits_df, pred_df, label_df], axis=1)
accuracy_df

,logit_0,logit_1,pred_label,true_label
0,0.476354,-0.056060,0,1
1,0.180653,0.148272,0,1
2,0.286365,0.200451,0,0
3,0.101160,-0.108817,0,1
4,0.178711,0.061986,0,1
...,...,...,...,...
172,0.305080,0.079071,0,1
173,0.267367,0.134212,0,1
174,0.162382,-0.067926,0,1
175,0.314368,0.100784,0,0
